In [1]:
# using Conda; Conda.add("scipy")
using Plots
using PyCall
pyplot()
n_channels = 19
file = "eeg.txt"
signals = readdlm(joinpath(pwd(),"exapmle",file),'\t',header=true)
sig = convert.(Float64,signals[1][:,1:n_channels]);
header = signals[2][:,1:n_channels]
header = [i[1:end-5] for i in header]
sig1 = sig[:,1];
sig2 = sig[:,2];

  likely near /Users/abcsds/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/abcsds/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/abcsds/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/abcsds/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/abcsds/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/abcsds/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/abcsds/.julia/v0.6/IJulia/src/kernel.jl:31


In [19]:
@pyimport scipy.signal as pysig 
freqs, goal_coh = pysig.coherence(sig1,sig2)
_, goal_csd = pysig.csd(sig1,sig2)[2]
_, goal_welch1 = pysig.welch(sig1)[2]
_, goal_welch2 = pysig.welch(sig2)[2]

129-element Array{Float64,1}:
  3971.0       
 16724.0       
  9759.64      
  3137.91      
  2254.06      
  1730.6       
   909.201     
   654.512     
   601.189     
   450.737     
   309.529     
   276.449     
   248.409     
     ⋮         
     0.00725959
     0.00731601
     0.00744099
     0.00707733
     0.00698235
     0.00684018
     0.00648913
     0.0068687 
     0.0071424 
     0.00697542
     0.00652593
     0.00349859

In [20]:
goal_coh == goal_welch2

false

In [21]:
goal_coh == goal_welch1

false

In [22]:
goal_coh == goal_csd

false

In [23]:
function cov2coh!{T<:Real}(out::Union{AbstractMatrix{Complex{T}}, AbstractMatrix{T}},
                           X::AbstractVecOrMat, Y::AbstractVecOrMat,
                           Xtmp::AbstractVecOrMat{T}, Ytmp::AbstractVecOrMat{T},
                           XYc::Union{AbstractMatrix{Complex{T}}, AbstractMatrix{T}}=out,
                           f=Base.IdFun())
    
#     out, Ac_mul_A!(work, X), Base.AbsFun()
    
#     (size(XYc, 1) == length(Xtmp) && size(XYc, 2) == length(Ytmp)) ||
#         error(DimensionMismatch("work size mismatch"))
#     (size(out, 1) == size(XYc, 1) && size(out, 2) == size(XYc, 2)) ||
#         error(DimensionMismatch("output size mismatch"))

    sumabs2!(Xtmp, X)
    sumabs2!(Ytmp, Y)
    for i = 1:length(Xtmp)
        Xtmp[i] = 1/sqrt(Xtmp[i])
    end
    for i = 1:length(Ytmp)
        Ytmp[i] = 1/sqrt(Ytmp[i])
    end

    for j = 1:size(XYc, 2), i = 1:size(XYc, 1)
        @inbounds out[i, j] = f(XYc[i, j])*(Xtmp[i]*Ytmp[j])
    end
    out
end

cov2coh! (generic function with 3 methods)

In [24]:
using StatsBase

In [27]:
cov2coh!(crosscov(sig1,sig2))

LoadError: [91mMethodError: no method matching cov2coh!(::Array{Float64,1})[0m
Closest candidates are:
  cov2coh!([91m::Union{AbstractArray{Complex{T<:Real},2}, AbstractArray{T<:Real,2}}[39m, [91m::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T[39m, [91m::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T[39m, [91m::Union{AbstractArray{T<:Real,1}, AbstractArray{T<:Real,2}}[39m, [91m::Union{AbstractArray{T<:Real,1}, AbstractArray{T<:Real,2}}[39m) where T<:Real at In[23]:6
  cov2coh!([91m::Union{AbstractArray{Complex{T<:Real},2}, AbstractArray{T<:Real,2}}[39m, [91m::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T[39m, [91m::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T[39m, [91m::Union{AbstractArray{T<:Real,1}, AbstractArray{T<:Real,2}}[39m, [91m::Union{AbstractArray{T<:Real,1}, AbstractArray{T<:Real,2}}[39m, [91m::Union{AbstractArray{Complex{T<:Real},2}, AbstractArray{T<:Real,2}}[39m) where T<:Real at In[23]:6
  cov2coh!([91m::Union{Abstr

In [30]:
C_xy = abs.(fft(crosscov(sig1,sig2))).^2
C_xx = fft(crosscov(sig1,sig1))
C_yy = fft(crosscov(sig2,sig2))

coh_est = C_xy./(C_xx.*C_yy)

105-element Array{Complex{Float64},1}:
 0.841597-0.0im      
 0.950705-0.0569581im
 0.803106-0.096577im 
 0.845085-0.15336im  
 0.664635-0.162196im 
 0.713418-0.22006im  
 0.581231-0.21814im  
 0.654168-0.291254im 
 0.442543-0.229673im 
 0.593798-0.354778im 
 0.410265-0.279714im 
 0.489445-0.378481im 
 0.358954-0.313609im 
         ⋮           
 0.358954+0.313609im 
 0.489445+0.378481im 
 0.410265+0.279714im 
 0.593798+0.354778im 
 0.442543+0.229673im 
 0.654168+0.291254im 
 0.581231+0.21814im  
 0.713418+0.22006im  
 0.664635+0.162196im 
 0.845085+0.15336im  
 0.803106+0.096577im 
 0.950705+0.0569581im